In [3]:
# NAIVE BAYESSSSSSSS

# standard
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder


data = pd.read_csv('NASCOTUS_fixed.csv')

# splitttering
X = data.drop(['justicesDecision', 'presAffiliation'], axis=1)
y = data['justicesDecision']


# 1 hot encoddeing
categorical_cols = X.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_cols]))
X_encoded.columns = encoder.get_feature_names_out(categorical_cols)


X = X.drop(categorical_cols, axis=1)
X = pd.concat([X, X_encoded], axis=1)


# train _ test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# yas 
class_counts = y_train.value_counts()
class_prior = class_counts / len(y_train)

# weights
class_weights = [class_prior[0], class_prior[1]]  # this assumes 0 is majority class and 1 is minority
# creating and training GNB using class weighting
naive_bayes = GaussianNB(priors=class_weights)
naive_bayes.fit(X_train, y_train)

# predciting
predictions = naive_bayes.predict(X_test)

# accuracy evaluation
accuracy = accuracy_score(y_test, predictions)
print(f'Test Accuracy: {accuracy}')


Test Accuracy: 0.8120862861147886


In [4]:
# I WAS GETTING AN issue where regardless of what I remove, the accuracy stays the same, so now I'm trying a new approach


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# data loading
data = pd.read_csv('NASCOTUS_fixed.csv')

# Separate features and target variable
X = data.drop(['justicesDecision'], axis=1)
y = data['justicesDecision']

# 1 hot encoddeing
categorical_cols = X.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_cols]))
X_encoded.columns = encoder.get_feature_names_out(categorical_cols)


X = X.drop(categorical_cols, axis=1)
X = pd.concat([X, X_encoded], axis=1)


# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# applying smote oversampling to the data....
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# train
naive_bayes = GaussianNB()
naive_bayes.fit(X_train_resampled, y_train_resampled)

#predicting
predictions = naive_bayes.predict(X_test)

# ACCURACY
accuracy = accuracy_score(y_test, predictions)
print(f'Test Accuracy: {accuracy}')


Test Accuracy: 0.4650728136437972


In [5]:
# I am checking for feature improtances

import numpy as np

# Get feature importances
feature_importances = np.abs(naive_bayes.theta_[1] - naive_bayes.theta_[0])  # Difference in mean values for each feature

# Sort feature importances
sorted_indices = np.argsort(feature_importances)[::-1]
sorted_features = X_train.columns[sorted_indices]

# Print feature importances
for feature, importance in zip(sorted_features, feature_importances[sorted_indices]):
    print(f"{feature}: {importance}")


lawType: 0.2128854229154169
certReason: 0.19700059988002394
presAffiliation: 0.04911817636472704
lcDispositionDirection: 0.0015116976604678811
